# Plotting the fitted maps

This notebook is to study and plot the fitted data.

It's aim is to:

- Plot figures
- Get line profiles from different properties of interest and see how they correlate


Import packages:

In [365]:
%matplotlib qt
#For pop-up window plots, with interactive functionality. If error, use instead %matplotlib tk 
import lumispy as lum
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
import addcopyfighandler
sns.set(style='whitegrid', context='notebook')

Use this code to load the fitted models and plot them.

In [277]:
root = os.path.abspath('G:\My Drive\PhD\projects\external_measurements')
session = os.path.relpath(r'20220118-CsAgBiBr3_2D_3D_films')

extension = "*fitted.hspy"

import os, glob
folder = os.path.join(root, session, 'fit_imgs')
session_path = os.path.join(folder, extension)

# For HYPMaps
paths_fits = [p for p in glob.glob(session_path, recursive=True)]
paths_fits.sort()
[('i={}'.format(i), os.path.basename(f)) for i,f in enumerate(paths_fits)]

[('i=0', 'HYP-MAP01_binned_11_2_fitted.hspy'),
 ('i=1', 'HYP-MAP02_binned_11_2_fitted.hspy'),
 ('i=2', 'HYP-MAP03_binned_11_2_fitted.hspy'),
 ('i=3', 'HYP-MAP04_binned_11_2_fitted.hspy'),
 ('i=4', 'HYP-MAP05_binned_11_2_fitted.hspy'),
 ('i=5', 'HYP-MAP06_binned_11_2_fitted.hspy'),
 ('i=6', 'HYP-MAP07_binned_11_2_fitted.hspy'),
 ('i=7', 'HYP-MAP08_binned_11_2_fitted.hspy'),
 ('i=8', 'HYP-MAP09_binned_11_2_fitted.hspy'),
 ('i=9', 'HYP-MAP10_binned_11_2_fitted.hspy'),
 ('i=10', 'HYP-MAP11_binned_11_2_fitted.hspy'),
 ('i=11', 'HYP-MAP12_binned_11_2_fitted.hspy'),
 ('i=12', 'HYP-MAP13_binned_11_2_fitted.hspy'),
 ('i=13', 'HYP-MAP14_binned_11_2_fitted.hspy'),
 ('i=14', 'HYP-MAP15_binned_11_2_fitted.hspy'),
 ('i=15', 'HYP-MAP16_binned_11_2_fitted.hspy'),
 ('i=16', 'HYP-MAP17_binned_11_2_fitted.hspy'),
 ('i=17', 'HYP-MAP18_binned_11_2_fitted.hspy'),
 ('i=18', 'HYP-MAP19_binned_11_2_fitted.hspy'),
 ('i=19', 'HYP-MAP20_binned_11_2_fitted.hspy'),
 ('i=20', 'HYP-MAP21_binned_11_2_fitted.hspy'),
 (

In [173]:
# Inspect one of the raw data and fit
n = 11
cl = hs.load(paths_fits[n], signal_type='CL_SEM')
m = cl.models.restore("gaus_fit")
m.plot(plot_components=True, vmax=10)
m.components

C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\misc\utils.py:1134: VisibleDeprecationWarning: The API of the `Polynomial` component will change in v2.0.To use the new API set `legacy=False`.
  warnings.warn(msg, VisibleDeprecationWarning)
C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\signal.py:3664: UserWarning: You are summing over a non-uniform axis. The result can not be used as an approximation of the integral of the signal. For this functionality, use integrate1D instead.
  warnings.warn("You are summing over a non-uniform axis. The result "


   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |          Perovskite |          Perovskite |          Expression
   1 |      broad_red_peak |      broad_red_peak |          Expression
   2 |          Polynomial |          Polynomial |          Polynomial

In [174]:
def make_mask_from_h_phase(model_comp, h_threshold=None, plot=False, q_h_threshold=0.1):
    h = model_comp.height.as_signal().data
    ncols= 2
    if h_threshold is None:
        h_threshold = np.quantile(h.flatten(), q=q_h_threshold)
        ncols +=1

    mask_bool = h > h_threshold
    if plot:
        f, axs = plt.subplots(ncols=ncols, figsize=(3*ncols, 3))
        axs[0].imshow(h, cmap='viridis')
        axs[1].imshow(mask_bool)
        if ncols != 2:
            sns.histplot(x=h.flatten(), ax=axs[2], color='C0', kde=True, bins=30)
            axs[2].set_title(f'Threshold: {h_threshold:.2f}')
            axs[2].vlines(h_threshold, ymin=0, ymax=500, color='C1')
        plt.tight_layout()
    return mask_bool

# Get one of the phase to mask the plots
cl = hs.load(paths_fits[n], signal_type='CL_SEM')
m = cl.models.restore("gaus_fit")
comp = m.components.Perovskite
mask = make_mask_from_h_phase(comp, plot=True)

C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\misc\utils.py:1134: VisibleDeprecationWarning: The API of the `Polynomial` component will change in v2.0.To use the new API set `legacy=False`.
  warnings.warn(msg, VisibleDeprecationWarning)


In [175]:
def make_gaus_parameters_maps(model_comp, cmaps = None, mask=None, sample_name=None, **kwargs):
    params = model_comp.parameters
    params_signals = [p.as_signal() for p in params]

    # Apply mask
    if mask is not None:
        for s in params_signals:
            s.data[~mask] = np.NaN

    params_means = [s.T.nanmean().data[0] for s in params_signals]

    headings = [p.name for p in params]
    titles = ["{}: {:.2f}".format(head, v) for head, v in zip(headings, params_means)]

    if len(cmaps) != len(params):
        cmaps = cmaps[:len(params)]

    hs.plot.plot_images(params_signals, label=titles, axes_decor='off', scalebar=[0],
                        cmap=cmaps,**kwargs)
    plt.suptitle(sample_name)
    plt.tight_layout()

# Get one of the plots
cl = hs.load(paths_fits[n], signal_type='CL_SEM')
sample_name = cl.metadata.General.title
m = cl.models.restore("gaus_fit")
comp1 = m.components.Perovskite
comp2 = m.components.broad_red_peak

cmaps = ['inferno', 'viridis', 'nipy_spectral_r']
vmax = ["75th", "75th", "75th"]
#vmin = [0, 0, 1]
make_gaus_parameters_maps(comp1, cmaps = cmaps, mask=None, sample_name=None, vmax=vmax,)
make_gaus_parameters_maps(comp2, cmaps = cmaps, mask=None, sample_name=None, vmax=vmax,)

C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\misc\utils.py:1134: VisibleDeprecationWarning: The API of the `Polynomial` component will change in v2.0.To use the new API set `legacy=False`.
  warnings.warn(msg, VisibleDeprecationWarning)


In [258]:
def compute_two_parameters(param1, param2, operation='ratio', clean_up= True, plot=True, histogram=False, **kwargs):
    s1, s2 = param1.as_signal(), param2.as_signal()
    label = ''
    ncols = 1
    if operation == 'ratio':
        s = s1 / s2
        label = '{} ({}) / {} ({})'.format(
            param1.component.name, param1.name, param2.component.name, param2.name)
    if operation != 'ratio':
        raise NotImplementedError(f"This operation ({operation}) is not yet implemented.")
    if clean_up:
        s.data[np.isinf(s)] = np.NaN
    if plot:
        f = plt.figure()
        hs.plot.plot_images(s, label=label, axes_decor='off', scalebar=[0], fig=f, **kwargs)
        # Add mean value box
        mean = s.T.nanmean().data[0]
        props = dict(boxstyle='round', facecolor='white', alpha=0.6)
        f.axes[0].text(0.02, 0.95, f'Mean: {mean:.2f}', transform=f.axes[0].transAxes, bbox=props)
    if histogram:
        ax_ins = f.axes[0].inset_axes([0.6, 0.05, 0.35, 0.2])
        # Filter data points which are 0 in the histogram
        hist_data = s.data.flatten()
        hist_data = hist_data[hist_data != 0]
        sns.histplot(x=hist_data, ax=ax_ins, color='C0', kde=True, bins=10)
        ax_ins.xaxis.tick_top()
        plt.tight_layout()

    return s

# Get one of the plots
cl = hs.load(paths_fits[n], signal_type='CL_SEM')
sample_name = cl.metadata.General.title
m = cl.models.restore("gaus_fit")

c1 = m.components.Perovskite.height
c2 = m.components.broad_red_peak.height

plt_args = {
'cmap' :'viridis',
'vmax' : "75th",
'vmin' : 0,
'suptitle': sample_name,
}
compute_two_parameters(c1, c2, histogram=True, clean_up=True, **plt_args)

C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\misc\utils.py:1134: VisibleDeprecationWarning: The API of the `Polynomial` component will change in v2.0.To use the new API set `legacy=False`.
  warnings.warn(msg, VisibleDeprecationWarning)
C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\signal.py:2247: RuntimeWarning: divide by zero encountered in true_divide
  getattr(sdata, op_name)(odata))


<Signal2D, title: height parameter of Perovskite component, dimensions: (|54, 54)>

## Process all maps

In [184]:
mask_signals_auto = False
save_folder = 'fit_imgs/fit_plots'

# Make new folder
try:
    os.mkdir(os.path.join(root, session, save_folder))
except:
    print('Folder exists!')
    pass

for path in paths_fits[:]:
    cl = hs.load(path, signal_type='CL_SEM')
    sample_name = cl.metadata.General.title
    m = cl.models.restore("gaus_fit")

    comp1 = m.components.Perovskite
    comp2 = m.components.broad_red_peak

    if mask_signals_auto:
        mask_comp1 = make_mask_from_h_phase(comp1, plot=False)
        mask_comp2 = make_mask_from_h_phase(comp2, plot=False)
    else:
        mask1, mask2 = None, None

    plt_args = {
        'cmaps' : ['inferno', 'viridis', 'nipy_spectral_r'],
        'vmax' : ["75th", "75th", "75th"],
        'suptitle': sample_name,
        }

    make_gaus_parameters_maps(comp1, mask=mask1, **plt_args)
    fname = f'{sample_name}_{comp1.name}_param_map.png'
    plt.savefig(os.path.join(root, session, save_folder, fname))
    plt.savefig(os.path.join(root, session, save_folder, fname.replace('png', 'svg')))
    plt.close()
    make_gaus_parameters_maps(comp2, mask=mask2, **plt_args)
    fname = f'{sample_name}_{comp2.name}_param_map.png'
    plt.savefig(os.path.join(root, session, save_folder, fname))
    plt.savefig(os.path.join(root, session, save_folder, fname.replace('png', 'svg')))
    plt.close()

    ratio1 = m.components.Perovskite.height
    ratio2 = m.components.broad_red_peak.height

    plt_args = {
    'cmap' :'viridis',
    'vmax' : "75th",
    'vmin' : 0,
    'suptitle': sample_name,
    }
    compute_two_parameters(ratio1, ratio2, histogram=True, clean_up=True, **plt_args)
    fname = f'{sample_name}_ratios_param_map.png'
    plt.savefig(os.path.join(root, session, save_folder, fname))
    plt.savefig(os.path.join(root, session, save_folder, fname.replace('png', 'svg')))
    plt.close()

    print(f'{sample_name} finished...')

print('Finished!')

Folder exists!
HYP-MAP01 finished...
HYP-MAP02 finished...
HYP-MAP03 finished...
HYP-MAP04 finished...
HYP-MAP05 finished...
HYP-MAP06 finished...
HYP-MAP07 finished...
HYP-MAP08 finished...
HYP-MAP09 finished...
HYP-MAP10 finished...
HYP-MAP11 finished...
HYP-MAP12 finished...
HYP-MAP13 finished...
HYP-MAP14 finished...
HYP-MAP15 finished...
HYP-MAP16 finished...
HYP-MAP17 finished...
HYP-MAP18 finished...
HYP-MAP19 finished...
HYP-MAP20 finished...
HYP-MAP21 finished...
HYP-MAP22 finished...
HYP-MAP23 finished...
HYP-MAP24 finished...
HYP-MAP25 finished...
HYP-MAP26 finished...
HYP-MAP27 finished...
HYP-MAP28 finished...
HYP-MAP29 finished...
HYP-MAP30 finished...
Finished!


C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\misc\utils.py:1134: VisibleDeprecationWarning: The API of the `Polynomial` component will change in v2.0.To use the new API set `legacy=False`.
  warnings.warn(msg, VisibleDeprecationWarning)
C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\misc\utils.py:1134: VisibleDeprecationWarning: The API of the `Polynomial` component will change in v2.0.To use the new API set `legacy=False`.
  warnings.warn(msg, VisibleDeprecationWarning)
C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\misc\utils.py:1134: VisibleDeprecationWarning: The API of the `Polynomial` component will change in v2.0.To use the new API set `legacy=False`.
  warnings.warn(msg, VisibleDeprecationWarning)
C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\misc\utils.py:1134: VisibleDeprecationWarning: The API of the `Polynomial` component will change in v2.0.To use the new API set `legacy=False`.
  warnings.warn(

# Plot spectra from ROI only

In [12]:
n = 1
c0 = 'blue'
c1 = 'green'
c2 = 'red'
use_fitted_data = True

cl = hs.load(paths_fits[n], signal_type='CL_SEM')
m = cl.models.restore("gaus_fit")
cl1 = m.as_signal()
print(cl1)

roi1 = hs.roi.RectangularROI(0,0,0.1,0.1)
roi2 = hs.roi.RectangularROI(0.1,0.1,0.2,0.2)

if use_fitted_data:
    cl_plot_1 = cl1
else:
    cl_plot_1 = cl

cl_plot_1.plot(cmap='viridis')

cl_roi1 = roi1.interactive(cl_plot_1, color=c1)
cl_roi2 = roi2.interactive(cl_plot_1, color=c2)

roi1_mean = hs.interactive(cl_roi1.mean,
                          event=roi1.events.changed,
                          recompute_out_event=None)

roi2_mean = hs.interactive(cl_roi2.mean,
                          event=roi2.events.changed,
                          recompute_out_event=None)

hs.plot.plot_spectra([roi1_mean, roi2_mean], color=[c1, c2])

C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\misc\utils.py:1134: VisibleDeprecationWarning: The API of the `Polynomial` component will change in v2.0.To use the new API set `legacy=False`.
  warnings.warn(msg, VisibleDeprecationWarning)
C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\signal.py:3664: UserWarning: You are summing over a non-uniform axis. The result can not be used as an approximation of the integral of the signal. For this functionality, use integrate1D instead.
  warnings.warn("You are summing over a non-uniform axis. The result "


<CLSEMSpectrum, title: HYP-MAP11 from fitted model, dimensions: (54, 54|511)>



ValueError: The value is out of the axis limits

# Load SEM images

In [349]:
extension = "*/*SE_Scan_*-SE.sur"

import os, glob
folder = os.path.join(root, session)
session_path = os.path.join(folder, extension)

# For HYPMaps
paths_sems = [p for p in glob.glob(session_path, recursive=True)]
paths_sems.sort()
[('i={}'.format(i), os.path.dirname(f).split('\\')[-1]) for i,f in enumerate(paths_sems)]

[('i=0', 'HYP-MAP01'),
 ('i=1', 'HYP-MAP02'),
 ('i=2', 'HYP-MAP03'),
 ('i=3', 'HYP-MAP04'),
 ('i=4', 'HYP-MAP05'),
 ('i=5', 'HYP-MAP06'),
 ('i=6', 'HYP-MAP07'),
 ('i=7', 'HYP-MAP08'),
 ('i=8', 'HYP-MAP09'),
 ('i=9', 'HYP-MAP10'),
 ('i=10', 'HYP-MAP11'),
 ('i=11', 'HYP-MAP12'),
 ('i=12', 'HYP-MAP13'),
 ('i=13', 'HYP-MAP14'),
 ('i=14', 'HYP-MAP15'),
 ('i=15', 'HYP-MAP16'),
 ('i=16', 'HYP-MAP17'),
 ('i=17', 'HYP-MAP18'),
 ('i=18', 'HYP-MAP19'),
 ('i=19', 'HYP-MAP20'),
 ('i=20', 'HYP-MAP21'),
 ('i=21', 'HYP-MAP22'),
 ('i=22', 'HYP-MAP23'),
 ('i=23', 'HYP-MAP24'),
 ('i=24', 'HYP-MAP25'),
 ('i=25', 'HYP-MAP26'),
 ('i=26', 'HYP-MAP27'),
 ('i=27', 'HYP-MAP28'),
 ('i=28', 'HYP-MAP29'),
 ('i=29', 'HYP-MAP30')]

In [350]:
sems = []
for p in paths_sems:
    name = "{}_SE".format(os.path.dirname(p).split('\\')[-1])
    s = hs.load(p)
    s.metadata.General.title = name
    s = s.isig[6:-4,6:-4]
    # axs = s.axes_manager.signal_axes
    # for ax in axs:
    #     ax.offset += 0.0
    sems.append(s)

sems[:3]

[<Signal2D, title: HYP-MAP01_SE, dimensions: (|54, 54)>,
 <Signal2D, title: HYP-MAP02_SE, dimensions: (|54, 54)>,
 <Signal2D, title: HYP-MAP03_SE, dimensions: (|54, 54)>]

In [351]:
# Check that both axes_manager for signal are the same
sems[n].axes_manager, cl.axes_manager

(<Axes manager, axes: (|54, 54)>
             Name |   size |  index |  offset |   scale |  units 
 ================ | ====== | ====== | ======= | ======= | ====== 
 ---------------- | ------ | ------ | ------- | ------- | ------ 
                x |     54 |      0 |     0.3 |    0.05 |     um 
                y |     54 |      0 |     0.3 |    0.05 |     um ,
 <Axes manager, axes: (54, 54|511)>
             Name |   size |  index |  offset |   scale |  units 
 ================ | ====== | ====== | ======= | ======= | ====== 
            Width |     54 |      0 |     0.3 |    0.05 |     um 
           Height |     54 |      0 |     0.3 |    0.05 |     um 
 ---------------- | ------ | ------ | ------- | ------- | ------ 
           Energy |    511 |      0 | non-uniform axis |     eV )

# Line scan from specific sample, component and ROI

In [403]:
n = 13
# Get one of the plots
cl = hs.load(paths_fits[n], signal_type='CL_SEM')
sample_name = cl.metadata.General.title
sem = sems[n]
m = cl.models.restore("gaus_fit")

c1 = m.components.Perovskite.height
c2 = m.components.broad_red_peak.height

c12 = compute_two_parameters(c1, c2, plot=False, clean_up=True)
sem.plot()
line = hs.roi.RectangularROI(left=0.44928, top=1.89696, right=2.94528, bottom=1.94688)
line.interactive(sem)

C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\misc\utils.py:1134: VisibleDeprecationWarning: The API of the `Polynomial` component will change in v2.0.To use the new API set `legacy=False`.
  warnings.warn(msg, VisibleDeprecationWarning)
C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\signal.py:2247: RuntimeWarning: divide by zero encountered in true_divide
  getattr(sdata, op_name)(odata))
C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\signal.py:2247: RuntimeWarning: invalid value encountered in true_divide
  getattr(sdata, op_name)(odata))


<Signal2D, title: HYP-MAP14_SE, dimensions: (|50, 1)>

In [404]:
print(line)

RectangularROI(left=0.398701, top=1.5948, right=2.89058, bottom=1.64464)


In [408]:
f, axs = plt.subplots(figsize=(7,6), nrows=2, sharex=True, gridspec_kw={'height_ratios': [2, 1]})

# Get data
c12_data = line(c12).nanmean(axis=1).data
c1_data = line(c1.as_signal()).nanmean(axis=1).data
c2_data = line(c2.as_signal()).nanmean(axis=1).data
x = line(c12).nanmean(axis=1).axes_manager.signal_axes[0].axis
x_units = line(c12).nanmean(axis=1).axes_manager.signal_axes[0].units

se_data = line(sem).nanmean(axis=1).data

# Normalise data
def normalise(data):
    return (data - np.nanmin(data)) / (np.nanmax(data) - np.nanmin(data))

axs[0].scatter(x, normalise(c12_data), c='C0', label='Ratio Pvk/Red')
axs[0].plot(x, normalise(c1_data), '-', c='C1', label='Perovskite')
axs[0].plot(x, normalise(c2_data), '-', c='C2', label='Red phase')
axs[0].legend(loc='upper right')
axs[0].set_ylabel('Normalised CL intensity')

axs[1].plot(x, normalise(se_data),c='k', label='SE signal')
axs[1].legend(loc='upper right')
axs[1].set_xlabel(f'Distance ({x_units})')
axs[1].set_ylabel('Normalised SE')

f.suptitle(sample_name)
plt.tight_layout()

# Plot the SEM with ROI on top
sem.plot()
line.interactive(sem, color='red')
plt.tight_layout()
plt.gca().set_title(str(line), fontsize='x-small')
print(line)

C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\hyperspy\signal.py:3616: RuntimeWarning: Mean of empty slice
  function(self.data, axis=ar_axes,))


RectangularROI(left=0.398701, top=1.5948, right=2.89058, bottom=1.64464)
